In [1]:
#Import
from spotify import Client, OAuth
import pandas as pd

In [2]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [3]:
spotifyClient = Client(auth)

In [4]:
spotifyAPI = spotifyClient.api

In [5]:
def getTrackAttributes(trackIdList = []):
    print(trackIdList)
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = filter(None, trackList)
    trackList = [x for x in trackList if x is not None]
    return trackList

In [6]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [7]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    #tracks = list(filter(None, tracks))
    return tracks


In [8]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        if trackObj['id'] != None:
            flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [9]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF
    
    

In [57]:
res = pipeLineFunc('2kpoUUJ5a4Cw3feTkFJhZ2','1276640268','top100')
res

['79jX8RM5CgPqDKdGEKNW9K', '5lGRrpyncZwEXXs4O5vGt8', '2XW4DbS6NddZxRPm5rMCeY', '2ekn2ttSfGqwhhate0LSR0', '3Vo4wInECJQuz9BIBMOu8i', '0sYm3c5chysXgZZSe3OQGc', '1L3NV7VrCiuE8C5QlhdeQL', '18mLuiwOxUI8wjMlzxRXsq', '0xM58jiyVWELjrR6rdNLku', '5hM6sP0Gh1jD57drszNueC', '7wvYmO5QscgIDDT0eNwbYX', '0ofbQMrRDsUaVKq2mGLEAb', '0QdLftqCLEADnbRPAKimHk', '4PpuH4mxL0rD35mOWaLoKS', '71T7xF5wLbDRg76jxAwDqA', '6s0GWZmWntEqjjzCDaVgIA', '0c21wrimkYpYS8ueXL9rrB', '1OAh8uOEOvTDqkKFsKksCi', '1bhUWB0zJMIKr9yVPrkEuI', '0mel2N9Ws9r4yLQn5QE21Y', '5oO3drDxtziYU2H1X23ZIp', '31MkGwj0SWgZaTvEt5Hgxv', '2swp6CpeAqQy8a3JDu9pBd', '2IKZkwHI5zsZf70STuhvOX', '6ZNo7Vi0TE9ul1fhKd4S1M', '09ts3GnICqYEU5PkQCpJK3', '45498paIaNnX2YvyeyR8IB', '7FESgYLcOufHqKlwrvJN60', '4Z6YruMINNspGet1gQyYlK', '2UVbBKQOdFAekPTRsnkzcf', '2x0WlnmfG39ZuDmstl9xfX', '1j4kHkkpqZRBwE0A4CN4Yv', '2bllegdYt2WoYdbRZyJ730', '2MMvonKGALz6YOJwaKDO3q', '7iNIg7XDEaYECfWD5dJ9Va', '4NzenFNqsYRvrG7rLdF1Ej', '0as7lPxkBPctZ7uVmchRNM', '3KAzXp6bTKmBvJFMLDXr2U', '7vGuf3Y35N

,album,artist,genre,id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,In My Blood,Shawn Mendes,top100,79jX8RM5CgPqDKdGEKNW9K,In My Blood,0.0524,0.628,211600,0.715,0,5,0.119,-7.097,1,0.0541,139.991,4,0.429
1,Dos Minutos,DULCE LÓPEZ,top100,5lGRrpyncZwEXXs4O5vGt8,Dos Minutos,0.393,0.588,245380,0.798,0,6,0.109,-4.126,1,0.0615,155.713,4,0.698
2,Scary Hours,Drake,top100,2XW4DbS6NddZxRPm5rMCeY,God's Plan,0.0244,0.753,198960,0.454,5.62e-05,7,0.498,-9.488,1,0.0963,77.17,4,0.344
3,Dua Lipa (Deluxe),Dua Lipa,top100,2ekn2ttSfGqwhhate0LSR0,New Rules,0.00256,0.771,208827,0.696,9.71e-06,9,0.179,-6.258,0,0.0755,116.054,4,0.656
4,Finesse (Remix) [feat. Cardi B],Bruno Mars,top100,3Vo4wInECJQuz9BIBMOu8i,Finesse (Remix) [feat. Cardi B],0.0185,0.704,217289,0.859,0,5,0.0215,-4.877,0,0.0996,105.115,4,0.926
5,La Loca No Soy Yo,Kenya Saiz,top100,0sYm3c5chysXgZZSe3OQGc,Guerra Mundial,0.334,0.614,248320,0.958,0,4,0.31,-3.833,1,0.0816,135.009,4,0.413
6,Filthy,Justin Timberlake,top100,1L3NV7VrCiuE8C5QlhdeQL,Filthy,0.0354,0.745,293947,0.579,0.0118,1,0.246,-5.771,1,0.138,97.002,4,0.645
7,"Ay, Ay, Ay Amor (En Vivo)",Calo,top100,18mLuiwOxUI8wjMlzxRXsq,"Ay, Ay, Ay Amor - En Vivo",0.257,0.394,224387,0.979,9.32e-06,7,0.864,-1.578,0,0.18,100.874,4,0.605
8,Me Largo,Dai Liparoti,top100,0xM58jiyVWELjrR6rdNLku,Me Largo,0.0101,0.47,197722,0.521,0,6,0.0863,-9.548,0,0.0515,135.261,4,0.152
9,"For You (Fifty Shades Freed) [From ""Fifty Shad...",Liam Payne,top100,5hM6sP0Gh1jD57drszNueC,For You (Fifty Shades Freed) (& Rita Ora),0.0188,0.483,244293,0.799,0,2,0.226,-4.567,0,0.0366,112.946,4,0.171


In [58]:
res.to_csv('billboardTop100.csv')

In [35]:
res.columns

Index(['album', 'artist', 'genre', 'id', 'track_name', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence'],
      dtype='object')

In [59]:
playListData = [{'user':'spotify','playlist_id':'37i9dQZF1DWTwnEm1IYyoj','genre':'Pop'},
{'user':'1154148597','playlist_id':'03Wky4J3HehgLEqgkYmia6','genre':'Pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5CHqBODmn2c','genre':'Pop'},
{'user':'1123398699','playlist_id':'23o2085ZcSb5V3yVxnHj9c','genre':'Pop'},
{'user':'1180623169','playlist_id':'2yTNGJu0CWKIms9vhpOPeY','genre':'Pop'},
{'user':'1288831866','playlist_id':'0lvwSeY2Og8XyileVyWZ1v','genre':'Pop'},
{'user':'dominic.rupp','playlist_id':'2zu4FqqzubZK4oXAZFEL4L','genre':'Pop'},
{'user':'redmusiccompany','playlist_id':'3U25AXqlklo3o6LqM1wX1I','genre':'Country'},
{'user':'1276364988','playlist_id':'7fIdjWhWzPbf1g5OSDKOXw','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX13ZzXoot6Jc','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX68H8ZujdnN7','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWZn9s1LNKPiM','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWWwzidNQX6jx','genre':'Rock'},
{'user':'sheffield806','playlist_id':'45vvbQl0GL7Niubb3H0U2Y','genre':'Rock'},
{'user':'redmusiccompany','playlist_id':'3yhmzZlOnRMCFV7PuMFPKy','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DX12YIlzNkccL','genre':'Classical'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8qqIDAkKiQg','genre':'Classical'},
{'user':'1269890947','playlist_id':'6Jn7zgkmA29infIRmptveP','genre':'Classical'},
{'user':'merovingin','playlist_id':'0ZRwrJ2EDGyKR6YgQPWXeO','genre':'Hip Hop'},
{'user':'akime1212','playlist_id':'2eY9lPvBKkluNSpeYJo71a','genre':'Hip Hop'},
{'user':'19jconnell79','playlist_id':'5772HGqmp2E99GQo5tfmcJ','genre':'Hip Hop'},
{'user':'1113721529','playlist_id':'0t3Mn6bXtPNWWZK6x9DtBH','genre':'Hip Hop'},
{'user':'playstation_music','playlist_id':'2NoR0KhNZ8oZnC3HoNt2FV','genre':'Hip Hop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8GjsySWIS1x','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DWYnY33e7En9e','genre':'Gaming'},
{'user':'1274440778','playlist_id':'5fdipI1K5g6ij4gCP8SvLj','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX6AIf8KmeDe7','genre':'Gaming'},
{'user':'emma.ehrhardt','playlist_id':'0QApAL6CpODIMyweRdAvM4','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX0SM0LYsmbMT','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVqfgj8NZEp1','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DX7YCknf2jT6s','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVzZlRWgqAGH','genre':'Jazz'},
{'user':'starbucks','playlist_id':'2UPjWBRjJqMQY3LRV2aDTS','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWSiyIBdVQrkk','genre':'reggae'},
{'user':'boogalooboo','playlist_id':'4gRio043A2No9SRU7ERq6q','genre':'reggae'},
{'user':'constantinex','playlist_id':'0s8tfHVdQa8duFMev3ZlgD','genre':'reggae'},
{'user':'constantinex','playlist_id':'53iDeoyHodextygU8wpras','genre':'reggae'},
{'user':'jmelendez1024','playlist_id':'0ka9e6tQIfLXVSNXraZxNz','genre':'reggae'},
{'user':'spotify','playlist_id':'37i9dQZF1DX2ohL85TE8TI','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DXbirtHQBuwCo','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5eq3AONkaho','genre':'K-pop'},
{'user':'thehela1997','playlist_id':'4c1DULvTgCGWcTETJrra5A','genre':'K-pop'},
{'user':'12165116393','playlist_id':'3OWkXYSwYzKeN64mCqkouR','genre':'K-pop'}]

In [60]:
playlistDF = pd.DataFrame(playListData)
print(playlistDF)

        genre             playlist_id               user
0         Pop  37i9dQZF1DWTwnEm1IYyoj            spotify
1         Pop  03Wky4J3HehgLEqgkYmia6         1154148597
2         Pop  37i9dQZF1DX5CHqBODmn2c            spotify
3         Pop  23o2085ZcSb5V3yVxnHj9c         1123398699
4         Pop  2yTNGJu0CWKIms9vhpOPeY         1180623169
5         Pop  0lvwSeY2Og8XyileVyWZ1v         1288831866
6         Pop  2zu4FqqzubZK4oXAZFEL4L       dominic.rupp
7     Country  3U25AXqlklo3o6LqM1wX1I    redmusiccompany
8     Country  7fIdjWhWzPbf1g5OSDKOXw         1276364988
9     Country  37i9dQZF1DX13ZzXoot6Jc            spotify
10       Rock  37i9dQZF1DX68H8ZujdnN7            spotify
11       Rock  37i9dQZF1DWZn9s1LNKPiM            spotify
12       Rock  37i9dQZF1DWWwzidNQX6jx            spotify
13       Rock  45vvbQl0GL7Niubb3H0U2Y       sheffield806
14       Rock  3yhmzZlOnRMCFV7PuMFPKy    redmusiccompany
15  Classical  37i9dQZF1DX12YIlzNkccL            spotify
16  Classical  37i9dQZF1DX8qqID

In [38]:
playlistDF["playlist_id"][1]

'03Wky4J3HehgLEqgkYmia6'

In [65]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

['1NrJYpdAi7uosDRPmSYrsG', '7evPpWWu01Klxi0KlPxR17', '0CZ8lquoTX2Dkg7Ak2inwA', '1bhUWB0zJMIKr9yVPrkEuI', '5Db9VIdDsN5yu3Eu7CT0i4', '63SevszngYpZOwf63o61K4', '11bD1JtSjlIgKgZG2134DZ', '3CKCZ9pfwAfoMZlMncA1Nc', '1HNkqx9Ahdgi1Ixy2xkKkL', '2YlZnw2ikdb837oKMKjBkW', '3C2xQqHVkhA1Ht17SzPRke', '5uCax9HTNlzGybIStD3vDh', '0prNGof3XqfTvNDxHonvdK', '0fioLzGM8ngbD1w6fMmm45', '3cNjgVBKTJ1SvKhunrCdVy', '6AeQlMyRzvSl1nkFztZyKl', '7oAraczbemGSNWqnbkU87R', '4nVBt6MZDDP6tRVdQTgxJg', '34gCuhDGsG4bRPIf9bb02f', '1EzrEOXmMH3G43AXT1y7pA', '4kflIGfjdZJW4ot2ioixTB', '7DFNE7NO0raLIUbgzY2rzm', '3goSVuTt3fDYDP6kRnFwuL', '7wMq5n8mYSKlQIGECKUgTX', '6GU19qDSbCWw5HKwfYTgVq', '4qug3gWrzeTNm0GhOkugJV', '3U4isOIWM3VvDubwSI3y7a', '24lMtPOCzP5g4hrg3NklLa', '51ChrwmUPDJvedPQnIU8Ls', '6XQHlsNu6so4PdglFkJQRJ', '0aOluBqXYd0rFSCsgDyAWX', '31MkGwj0SWgZaTvEt5Hgxv', '7krbSH3rd8lhIZvuzTV3Bl', '75JFxkI2RXiU7L9VXzMkle', '4QfNOZN7vrQCQKK1ZJ3XPb', '4h0zU3O9R5xzuTmNO7dNDU', '3BgvCj5oUc8jpL8YCB9y8S', '37FXw5QGFN7uwwsLy8uAc0', '3XVBdLihbN

['39V2ov3LpC6BKteOWaFeYW', '24j6yidzOpNuLgWPIDw1Fl', '6DavkN0Z10Sgivh8PT5cbu', '7f7xRKukld8LH7aVAhJTQG', '0hBXjNTykXELxhdTxNe5no', '1rbxDbeC3JfN0uSchI85we', '4vlmiFfNIgdfuQJcIgIYhk', '5hYvTtAoBwZ4l4oO0A9Ntl', '1DihsSBztAD6qJ5TTEE90F', '218o2LbwBEtYDbDWcuUUxG', '3RokoLDR9CAiaXZX9dtJe0', '2o7We8msB5T5Ef36OccIIP', '0UmmSFh7Kpk50unE55OdOm', '2NeJZefE9rrbEq33I4wFDC', '3efQmQl7VA3ukppVtbzzMI', '045ZYGHDZYVvJwyZOWEXPs', '53i6D188bSUeVZeR1sMucH', '6WPDJLHlR2fFXNaRKq0jqm', '0sjSgTluUJIdqfyRe4EQ4U', '5wwxTp4LuwS7LfHMZvwoQn', '15dYrloTvmcrkZ7xv5mfID', '0vzjqLeM8F95Y7uYPVkESU', '0prwCihRoIg2yvGpdiNTKr', '3z5eL4hQXaBWxSWZUTwI4e', '0p3x5S3Ay0sVUQhjebIxDb', '1P1vtfVHyYTYhVX1is7qcT', '3R1sSj96zhzNsz8TEimG17', '4g8dSNhJAquLXiUOzw7MLo', '6jGo0omgO1v9Bx8CqZcKtk', '2mrBlPO3a14oHOMd3wJHnt', '7LUEySiNE50glN3t3LkvhP', '0ylvUTWJQiPIjtNrLQ4BpL', '2qZKzHdBJGrEVNlYV7LH6s', '5nBezR9uwtcJ6CSejEcl95', '1uV0cD0UenUlaq6SDpj2zh', '0jmrRnVetXvXlU6RUqnoTY', '2q62duXLlXrE4Nyixhr71a', '3dlqdb9HozhH2ThmxBfbWF', '542oFGL2mK

['498ZVInMGDkmmNVpSWqHiZ', '7mldq42yDuxiUNn08nvzHO', '5Toaot8f2kE2kmCSEPuiSn', '2lFtnEmkIPm2ClN55e2chV', '0iA1unTbTbDOWUSlbwJ1pS', '6YKL9gCOH31W5fanjQTtqq', '0a3KWFyJEaOBOGKjTGOoN8', '5Cbu6tK36mFasjGE9N7xrV', '5sjIhQzNljMVrDklI91ezp', '0ipcmHor30Y8p6hCd8QUVq', '63OQWdXMq0Pe6pRpWY5gO0', '7zVCrzzEJU7u24sbJPXA5W', '1WQ6RBe6WwXOljd2eUEjkn', '4fT0lwTnwXN5Qg5vSPyq0u', '571YdX5KyoLgsVmauUD37w', '6xHI9KjUjYT0FPtGO8Mxa1', '1N0j6Gw5lmfI7kipAgYcma', '4JjIIBNoho0V2gZjH4nEzR', '2Vjlr90bT7kJhB50n5ocBd', '7px4t7HGiuMUvFkEOpZEwp', '41EYTKX66PslehSVrFdT4p', '5Feu0AyHLrW0mCRAyUnZ6W', '6yop3isbCrce4maIoUdH0F', '38gZVjeaeR3gcnCzziAxBm', '0xwPzLmBAYro8BUz7MrtAo', '1gmQOq1lCIJJ3frQUVWbRH', '4imCYWWK0czzTErcVlA1EQ', '4D4xJ4LyDXBWIf9thFSMAJ', '0ECiofa54uAxOjtlKxGlwF', '5Zr5DLfEF2tCP5SNoxUmvV', '5kTlia5MsKQfaU75D0Ev2y', '2WaOQWSv0D1tZNxyLvnGAI', '6zDx6Xf99MZPziRZH2iddZ', '482Ln2gRE7epeptln1k3aH', '2UZFsGsNiDp6xd6zQ8vqlW', '4VFE6ZNqa8jHAmbYICoAFg', '6zLdXcmvIxnAXpz9sXFnIE', '577QJvQb5HSp3Fl9k63pgU', '7tLmMruWwj

['26gceVsA1ybT8qsX3aYLEn', '7F3uo3304R2CTvOseU3H7u', '1vNoA9F5ASnlBISFekDmg3', '7EUfNvyCVxQV3oN5ScA2Lb', '2Pvr8rYuh5J8V7QglJhH6W', '5s9ZfQasBGM8YLNAhXx7or', '7c71baHwrGDOeIFl1gVO14', '5zuY71mWmwarLSzzJHygP2', '5mdooqkfMvf7LtIFKReUYg', '4qSsjDGXplb6422X2INvFW', '4BdGO1CaObRD4La9l5Zanz', '3xXBsjrbG1xQIm1xv1cKOt', '39QwSstMn0mtFnJqkEK2cH', '7pV6zXFpCwrrmcosBAtp0o', '1bOUT9lpQltpePmnS6NM2a', '3e07PcEvFRAE21NIOuNFZ0', '7KlOrQFS5NZoAqDzJ2N7r4', '3ZrpYrx9TlulkmWVz4t9Tc', '64kDGEpOWlkv2l6x4TVNOs', '7nAfXgeHfDO50upcOjJOaq', '2ZIhuxgLILR8LGBIWevkEe', '7gHekxiG3v4Io7at2vDcJc', '7m0OBz7Y2xz3cyhB0uXiFx', '3kdMzXOcrDIdSWLdONHNK5', '35zrlBOjpfDPMcZzglWOuV', '60eOMEt3WNVX1m1jmApmnX', '3YQ96qkxiEwTLwgu2Izcmq', '6LzsmlN5UbCYytaaKZueRH', '0w5Bdu51Ka25Pf3hojsKHh', '2gUe8cQgJj5AfM2O5nuiwq', '3IqHzcXy7IsYYND6lKwPAi', '1qPlNM2FaEWCMSxpVR5XeH', '6JnRLa50GuHzF0ucg9AGMI', '4U9arGcc4ATa7shChhLQP0', '7dITaPcAWMIHwOtRvRjCsV', '7g58rb2dFDCvcmLBwpHR9F', '7jFjcIcqaRtIuz85eU0ahQ', '5adBpeh1CjHAzlM2i8nYFR', '3QuDdSIahx

['0IGqFsBp1eI5kxAReABcv2', '5cKQS9890SpoU2DyqclNWG', '4gEsgkvHO7MKMEVnBbQudj', '5eyIfIp6IafrdAjRnZF6ql', '1poK64Wt7p7VwrAbFgD5hS', '6Vas6h9lB5mmeVDXdFqlQA', '4hLDCpCrPzTZ2BqJJBJXEA', '7Ivn1AgtIVM708UdwKASbc', '7IwURvEfVcdxUCjLKUu6sv', '6dBQ5asGzaKH9313wBxvIz', '4voEoczU7Ijborps9XF1n3', '2RhDRSGU8YDt2HkOMWhQrf', '31WbCaQjJGV6lTNbDsUYXb', '0BdAjgQqEb12CyH19xBU2N', '3SR5YO0lKhCy5A5cTd3Xzo', '3JempQ95v2ScWFBAGko352', '7yQsd17sVkOLaDwa4tz9dT', '5cfjHBdMjf0x07VZD70MlE', '4Tttv4p2xuAq1LpQ7LI95E', '7HFl64GKBtPTa5YoW7j4jL', '4AE7Lj39VnSZNOmGH2iZaq', '6pNvKPvOknI5zI4o9TB43f', '5xJkY5PiDpInkBPtaArVax', '2qmxggATKFIeg68guRSG3r', '6nwAubYnLGl7F7cDuOoVLp', '3cAko9H1rfeIO1sGl18ru9', '4DcrZLp19qTdBYqc2bgj3V', '36TBMtrW4x7bHW8LH8rnRr', '51bnnsURQ9bITMraBp4Vbu', '3JaFo8M3MI74dQeKdDlNkM', '305jsaqcBPgO7FNnIyDmOE', '5ulZo7nk3MByXLvja3NCAc', '503OTo2dSqe7qk76rgsbep', '6KDHudERIEJMFAR76XJrZ1', '6eDwF11wieCPDzErj3bXwE', '5znkjQkB05nTyRs4S6ntHQ', '5PbFD3AIyunz5HxocduA9e', '3QHONiXGMGU3z68mQInncF', '7o7rrKyfqG

['2WfaOiMkCvy7F5fcp2zZ8L', '08mG3Y1vljYA6bvDt4Wqkj', '04XdCMi1WhAjVyRJbAtaq0', '2KH16WveTQWT6KOG9Rg6e2', '6RgGppm2SSX5p246UqfdEi', '6SjbzGo7arjfM5cXpXhc5B', '0Yt04VZEo0lBQrpX1FVfyY', '5ChkMS8OtdzJeqyybCc9R5', '54flyrjcdnQdco7300avMJ', '7N3PAbqfTjSEU1edb2tY8j', '5cP52DlDN9yryuZVQDg3iq', '57bgtoPSgt236HzfBOd8kj', '5nNmj1cLH3r4aA4XDJ2bgY', '1AhDOtG9vPSOmsWgNW0BEY', '57JVGBtBLCfHw2muk5416J', '1OOtq8tRnDM8kG2gqUPjAj', '1pKYYY0dkg23sQQXi0Q5zN', '3u9fHuAtjMY1RW2mZfO4Cf', '45Ia1U4KtIjAPPU7Wv1Sea', '5NIPsWpDjJTFBoPxCUUeXp', '40riOy7x9W7GXjyGp4pjAv', '3MrRksHupTVEQ7YbA0FsZK', '5ghIJDpPoe3CfHMGu71E6T', '5VVwwfK6Cp9IydkhKfJUEq', '6ZOBP3NvffbU4SZcrnt1k6', '5yEPxDjbbzUzyauGtnmVEC', '3gce83TvahSnFHFtCqbe4R', '66S14BkJDxgkYxLl5DCqOz', '5Q41NLTmGbVPozwHKK7bk2', '2374M0fQpWi3dLnB54qaLX', '1FTSo4v6BOZH9QxKc3MbVM', '6Sy9BUbgFse0n0LPA5lwy5', '1YTLK2GZMai3LjDwzDL9OD', '11LmqTE2naFULdEP94AUBa', '5RHG4SQ6iJ1qQJmt3EFJCX', '0XIvZ82aDF7JiSi3ZE320u', '028r3igL3M00VdRCe2Ga7Q', '0jWgAnTrNZmOGmqgvHhZEm', '7qQnBfwXrw

['4UWHQwPvAv3COsw8YUhYMY', '4QRibf66gbTMjouBJbUqGW', '2tu9v4F9BoQqknTPAozovw', '2SkrcT9HiguqwIzWenKqJP', '6xD2MrU32JmTLZkGyPIhrV', '6rHOZ7nkocvJGJ3RGa64ns', '7j0rbwwoWsK3FCJUMCr0xE', '7cl3yA2oecJ3ROiRxZtsdm', '5U0xbaYq8B55JwpEWWPorq', '2d64mGw2MDm5UL3nUzgr70', '7AyCstPRHjQctIhzhjUN3j', '3dt7sqYFnwFJHFncCIitwU', '2nOqZ2AMZLbe0rRWsRJc5y', '4AzvNQcn1I4sZ2wJrW5ATq', '3F0mcxksBp33QrL6oyjvLN', '3OpQVwRE5b8DpCEo0iPKTH', '75444x8p4Mrg5GCw2rB0VH', '3by8TFARD2faf2MjZbooSH', '3EPrsFoVjF15alyEGlQCpT', '6V0fnOISXTrs1lBI7uf3oo', '3QPcBMWD9E862ykD5MNWqs', '7MxMYU9BvE7RMrIUSokL29', '23gcQr3NRKzLXsP9H5jFQ1', '70zBHxvHa0CkmY3C9fVzNp', '1KxFIoNhFJNDTh6aBQq1QM', '6krakCJNLZNH0QwfBjjrgp', '3D8YvF6OEeJtszmKJOIfsM', '2Cl57cd1XuO9DW1ByR3jyD', '6ngyBaILAASovRDDxeohS1', '7o2SgrTlLry8e0pHJrdqQ5', '0Rs0yeWSSQb6cSbfmvUWrA', '3RUh9DrYDkDCUd1XELtuF0', '7HauobafjR3NDJxCl555qD', '1UxFRHpZ8CWhJQYw24N16o', '7vhXQtcsA4enprGM6QvJqc', '4D8UvvJUIapsaZZSLqXrKP', '3pb9alcNVn5gq9kCBtAL2e', '1cP4whyH8nbXq66JzCkzeI', '4CkYaaq1pF

['5NuNJOapnXYAJeeo3Q7cSr', '0Mv4w87XU3HtV412slUc1r', '4uOKFydzAejjSFqYbv1XPt', '7FILn9Akg2468HPNsRZCkq', '6NvTJWYPQ5RnLgmOK4spzq', '2e5Vm6VvdlTzuNcRSikeU6', '1vUBGvWcEiEFGeg94zl1Co', '0npA0d5jbB01k53LMRccKP', '48HwknWByfac0fdNyYyQDg', '5FgdPuK51WBYhuMhV0XIBg', '1ZPsdTkzhDeHjA5c2Rnt2I', '1nw8lTaFu9kphHRNy67lXa', '1N33JzVusZ8awiZo6kcldO', '1XRW0FW61bznuK3XDKFiKv', '5mWaxl8KzwAop3kfNNyNZX', '5ByIHT8s38diBQf6dkEWbt', '1Bx0zEdVjkFlV27iKaePug', '3sbBOn0ifcgoX3MfaTeFKr', '7bnRX1ObyGzzSuOWrNFLWw', '4efoEY8iDBzUqitjmNDhpN', '1fQxtJ2yrBn6l20VVuGuJF', '16DTl4nJDz444OcvTlbxxD', '2vDUqLZsaEEYmREcRo9gnn', '3j8EEMm9kbPLqByuaHmUmQ', '0mD19MC6H0OzHvEUADHypU', '7mV9ia2PsTJ1yYmGNv9tEE', '1VkNLfYxoS7Uc80xvt1xBK', '2PdAkRZKiwjDzRNWFdXvp6', '0URS9f6Ck8mzhou6AMNCr5', '4HWhdjFpP7Ki6GdpAmIfxK', '43VSZHjLjjxSi7KosnHZ8w', '1hCohpxXi6GDJQ85PN70ma', '2OHQhOiGIYaXqqJVIMpNrF', '1Rvl8qsKJurfFTyWLBI9ib', '6VPTyILdJ2NLs0KGhscS34', '2p5xGfnvZabKKSYT1HOU0E', '0lzOFW7ELjXom0Jua03stq', '6eXNZLPoIikMlKuo1lKBAa', '7usMsERH3s

['1HTV5zPNJkpWzQEIAQpzad', '7FAZYpMTElK5arLC452mzX', '5clFSlfkCRlhnH1cAQjSBi', '2r7mfzeXCzyrysTya4Onwk', '2htll8gP4tlYgrl0S2nJwv', '7jw1h8mwJLFz42UtthC6cU', '1YmY1z08Z8Xlfl4E7hSfuM', '40EDOraCqoiMf14TzODihf', '5eBgkIJ0UsMxsnXD5viRC4', '2OikDCslmF1Y04Szgbc8Hl', '0BlgEwW4l938kmdjknfxyW', '2vWWy37CDlDlfPipzb4iIX', '614NVFo5csQqRnryl4jXs2', '6sfBQReZH9v9tYSs82Xt1O', '45IOnEpVUdxWUZFqpZ7Xvz', '5YJ4BVvTFmVCxUyhKejHfR', '3XOcKSqWYaLXMjZHDz7YvK', '49vwUeeauJgEQxDViXA01C', '70kSMcgebeHkly6xGS2Xtm', '6TT2GzEieukcMFQkCWw2Tm', '40ireFCD2S01JkFz4EvMdQ', '4z7Xlm5zCZDHHdYN24MPN0', '5tAM9ZyV1NoRkSK0TzFcgP', '0Y90MBWxynAZaaFGMTw58B', '28QkqSjl1Pi0niDc20RDl0', '2y7EqBBJWdsha2YKjB5uHy', '1PZrHbJjt5t7dP9OGRxcD0', '5ZUIgN45lEqa8fTOWbh4yn', '2yj5LfhJGK0zhA2CZMgT6R', '5SgN4mSGVEZ8TZQ5YSsKKb', '581HUMQdWa3Y9dCpF8iatr', '0yy6SegPRLQU03ftqy8w6L', '03jLaxjIXNOh9TwNX0jKPk', '5Qy6blxPfXwDL8felSjilM', '1Q5QDlgl9zAQ4a2WeM4gQE', '4kAtRwNRLMKuoCbxMSxUGI', '04uA3BJ1Rj8DBKkpKCABJx', '0rz1ULgbOEAsqfe9A97tmI', '1Wqey5mexj

['1OqDvYVDhJyFZny7XlfIyZ', '2VVqxQhALecXvaArlZSVXr', '02sD9kxKrCboVcOKzx3VyZ', '6NbfFVZ01Ttvvqmcok8DxL', '5JwuXvMV8f7ftQczIPCzmm', '0ImOeRJIA9YL9gmSOeC544', '45uJULGOd9HoZi2aPySiwn', '4domD3GWhDFM6q9eqfjSsy', '4YAgH8s0JFfwikTNwe0aIk', '0ErzcmZ2gIwX7X0xSMQPix', '3TM7sUA8tMzndo6G8E526g', '1mdEvT7X3Rs7vL75fRb8oo', '3XzxHvEomjJIKZcZqYfFut', '3bNjjq8KpMx2Y9gpCoJl2u', '7cbZIBLhfD9taMBgEsIhIp', '5gA9Xn8oPts2aewPgxVkPD', '3PbsYQc7kMKFaJMs1Cj3tj', '5GKwq4sO5ZHKuWaDmdwMQc', '3BPoSr2pO34Aan6alFfVto', '1IvS2WRu87NUk8In1R1iuW', '4W0wSLXqn0L3XneeBC344O', '7jwEER43W8bYYsKUdl39Aa', '2892MswwqljFCzto12m7LE', '1dJQaGuotdQeGeI6gToWLc', '0slrpwbibfQNxpjqmgtkUh', '1PyetXTPlLlZAZaBvtwbYJ', '5SE57ljOIUJ1ybL9U6CuBH', '2pp23RYpdWNaqDrM67rNLw', '48Dw7SrfNs81WBIpEysOkd', '2sio5nPavoux62tH7i6x2H', '5WbnKef7BL5E3NCfnGpb2z', '5y51ta5LHTRLpPVxnn4zXZ', '19t5GSN3XsLB7UOsZD8Fwv', '5EzitieoPnjyKHAq0gfRMa', '1cgBWgoL6520lR2QZDzdGN', '1nIGvBzMykPmKjUJRWOwHE', '5pesNiBKAx8JNwK2mQ2HEc', '53YZi9zgTnF0Py0K6ejyWz', '4MSiPP89oZ

In [67]:
trackDFMain.to_csv('tracksInfo.csv')

In [47]:
res = pipeLineFunc('37i9dQZF1DWTwnEm1IYyoj','spotify','Pop')
trackDFMain = trackDFMain.append(res)


['1NrJYpdAi7uosDRPmSYrsG', '7evPpWWu01Klxi0KlPxR17', '0CZ8lquoTX2Dkg7Ak2inwA', '1bhUWB0zJMIKr9yVPrkEuI', '5Db9VIdDsN5yu3Eu7CT0i4', '63SevszngYpZOwf63o61K4', '11bD1JtSjlIgKgZG2134DZ', '3CKCZ9pfwAfoMZlMncA1Nc', '1HNkqx9Ahdgi1Ixy2xkKkL', '2YlZnw2ikdb837oKMKjBkW', '3C2xQqHVkhA1Ht17SzPRke', '5uCax9HTNlzGybIStD3vDh', '0prNGof3XqfTvNDxHonvdK', '0fioLzGM8ngbD1w6fMmm45', '3cNjgVBKTJ1SvKhunrCdVy', '6AeQlMyRzvSl1nkFztZyKl', '7oAraczbemGSNWqnbkU87R', '4nVBt6MZDDP6tRVdQTgxJg', '34gCuhDGsG4bRPIf9bb02f', '1EzrEOXmMH3G43AXT1y7pA', '4kflIGfjdZJW4ot2ioixTB', '7DFNE7NO0raLIUbgzY2rzm', '3goSVuTt3fDYDP6kRnFwuL', '7wMq5n8mYSKlQIGECKUgTX', '6GU19qDSbCWw5HKwfYTgVq', '4qug3gWrzeTNm0GhOkugJV', '3U4isOIWM3VvDubwSI3y7a', '24lMtPOCzP5g4hrg3NklLa', '51ChrwmUPDJvedPQnIU8Ls', '6XQHlsNu6so4PdglFkJQRJ', '0aOluBqXYd0rFSCsgDyAWX', '31MkGwj0SWgZaTvEt5Hgxv', '7krbSH3rd8lhIZvuzTV3Bl', '75JFxkI2RXiU7L9VXzMkle', '4QfNOZN7vrQCQKK1ZJ3XPb', '4h0zU3O9R5xzuTmNO7dNDU', '3BgvCj5oUc8jpL8YCB9y8S', '37FXw5QGFN7uwwsLy8uAc0', '3XVBdLihbN

In [77]:
uniquetrackDF = trackDFMain.drop_duplicates('id')

In [79]:
uniquetrackDF.to_csv('tracksMusicInfo.csv')

In [15]:
spotifyClient.auth.token

{'access_token': 'BQCIEwW0OfoFhOjq-Komou_V5O034CzZYVv2LobYCMXXWRAWYg2Ybac_J-xtW4FqIfmDPr-orAAjeHQ94bU',
 'expires_at': 1524761869,
 'expires_in': 3600,
 'scope': '',
 'token_type': 'Bearer'}

In [2]:
import pygn

clientID = '740529234-9D358964C751528ED8F5DA9B0D060C84' # Enter your Client ID here
userID = pygn.register(clientID)

In [ ]:
userID

In [5]:
metadata = pygn.search(clientID=clientID, userID='varadtupe', artist='Kings Of Convenience', album='Riot On An Empty Street', track='Homesick')

In [6]:
metadata